In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

2024-10-19 20:37:40.286232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 20:37:40.922416: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 20:37:41.344001: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 20:37:43.333308: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-19 20:37:48.346723: W tensorflow/compiler/tf2

In [7]:
# Load data
df = pd.read_csv("processed_stock_data.csv")

In [8]:
# features to use
features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [9]:
# Normalize 
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])

In [10]:
# Create sequence for forecasting
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length), :])
        y.append(data[i + sequence_length, 3])  # Predicting 'Close' price
    return np.array(X), np.array(y)

In [11]:
sequence_length = 10  
X, y = create_sequences(scaled_data, sequence_length)

In [12]:
# Train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
## Define LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

2024-10-19 20:39:31.355000: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79078 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:b7:00.0, compute capability: 8.0
/blue/yonghui.wu/y.khan/conda/envs/hf-llm/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


I0000 00:00:1729384802.022309 2549962 service.cc:146] XLA service 0x1521ac005170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729384802.023113 2549962 service.cc:154]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-10-19 20:40:04.206626: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-19 20:40:08.346697: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


26/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1248

I0000 00:00:1729384813.719251 2549962 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 165ms/step - loss: 0.1171 - val_loss: 0.0022
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0020 - val_loss: 5.8220e-04
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.1574e-04 - val_loss: 5.6367e-04
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.8200e-04 - val_loss: 5.0698e-04
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.4368e-04 - val_loss: 5.2201e-04
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.0372e-04 - val_loss: 5.0781e-04
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.6934e-04 - val_loss: 5.0978e-04
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.7345e-04 - val_loss: 4.8281e-04
Epoch 9/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.2815e-04 - val_loss: 5.0703e-04
Epoch 10/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.0042e-04 - val_loss: 4.8930e-04
Epoch 11/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.0409e-04 - val_loss: 5.0108e-04
Epoch 

In [14]:
# Mean squared error
mse = model.evaluate(X_test, y_test, verbose=0)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.00023729885288048536


In [15]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(np.hstack([X_test[:, -1, :3], predictions, X_test[:, -1, 4:]]))[:, 3]
actual = scaler.inverse_transform(np.hstack([X_test[:, -1, :3], y_test.reshape(-1, 1), X_test[:, -1, 4:]]))[:, 3]

# Calculate RMSE
rmse = np.sqrt(np.mean((predictions - actual) ** 2))
print(f'Root Mean Squared Error: {rmse}')

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Root Mean Squared Error: 1.6676921664236952


In [16]:
######### with Sentiment
features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sentiment']
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length), :])
        y.append(data[i + sequence_length, 3])  # Predicting 'Close' price
    return np.array(X), np.array(y)

sequence_length = 10  
X, y = create_sequences(scaled_data, sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/50


/blue/yonghui.wu/y.khan/conda/envs/hf-llm/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - loss: 0.1789 - val_loss: 0.0059
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0030 - val_loss: 0.0014
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0013 - val_loss: 7.8862e-04
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.3157e-04 - val_loss: 6.4598e-04
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.4743e-04 - val_loss: 6.1377e-04
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.0790e-04 - val_loss: 5.3265e-04
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.6524e-04 - val_loss: 5.4848e-04
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.3681e-04 - val_loss: 5.0738e-04
Epoch 9/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.3111e-04 - val_loss: 5.1869e-04
Epoch 10/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.1227e-04 - val_loss: 4.7137e-04
Epoch 11/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.3573e-04 - val_loss: 4.8423e-04
Epoch 12/50
29/2

In [18]:
mse = model.evaluate(X_test, y_test, verbose=0)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.0002803955867420882


In [19]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(np.hstack([X_test[:, -1, :3], predictions, X_test[:, -1, 4:]]))[:, 3]
actual = scaler.inverse_transform(np.hstack([X_test[:, -1, :3], y_test.reshape(-1, 1), X_test[:, -1, 4:]]))[:, 3]

# Calculate RMSE
rmse = np.sqrt(np.mean((predictions - actual) ** 2))
print(f'Root Mean Squared Error: {rmse}')

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Root Mean Squared Error: 1.8128156284083992
